#### Lecture 1. Quantization

- exploitation of precisions

#### mini code : floating-point operation

In [6]:
print(0.1 + 0.2, 0.1 + 0.2 == 0.3)
print(0.1 + 0.3, 0.1 + 0.3 == 0.3)

sum = 0
print(type(sum))
for _ in range(7):
    sum += 1.0 / 7.0
print(sum, sum == 1.0)
print(type(sum))

0.30000000000000004 False
0.4 False
<class 'int'>
0.9999999999999998 False
<class 'float'>


1. Fixed point, floating point
- fixed point : sign + integer part + fractional part
- floating point : sign + exponent(지수부) + mantissa(가수부)
    - 더 넓은 범위의 숫자 처리 가능, FPU 필수, 숫자 정보량은 동일(동일-bit라면)
- continous value, discrete value
- precision(정밀도, variance 개념) / accuracy(정확도, bias 개념)

2. Quantization 이란
- model size 감소, inference speed 를 높이기 위한 기술, lossy conversion
- affine quantization : 직선 상의 두 점의 거리 비를 보존하는 affine 변환
- active function을 quantize 시에 역전파 미분이 안되는 문제
    - $\frac{dy}{dx} = 1$로 가정, smoothing 된 계단 함수 사용

3. Quantization 예시
- how much : 16bit, 8bit, 4bit,.. , or mixed-precision
- how to : dynamic(activation이 inference 도중 양자화 / static
- when : post-training(training 후 weight 값들에 대해 양자화 진행) / quantization-aware training(학습 진행 시점에 inference 시 양자화 적용 영향 simulation)
- what to : weight, activation
- harware aware quantization

#### Lecture 2. Knowledge distillation

##### mini code : (hard) max, argmax, softmax

In [8]:
import numpy as np
DATA = [1,3,2]
ans = np.max(DATA)
arg_ans = np.argmax(DATA)
print('max: {} VS argmax: {}'.format(ans, arg_ans))

p1 = np.exp(1) / (np.exp(1) + np.exp(2) + np.exp(3))
p2 = np.exp(2) / (np.exp(1) + np.exp(2) + np.exp(3))
p3 = np.exp(3) / (np.exp(1) + np.exp(2) + np.exp(3))
print('DATA 원소들의 softmax 전체 합 : {:.1f}'.format(p1+p2+p3))
print('DATA 원소들의 softmax 값 : {:.4f}, {:.4f}, {:.4f}'.format(p1,p2,p3))

from scipy.special import softmax
scipy_ans = softmax(DATA)
print('DATA 원소들의 softmax 전체 합 : {:.1f}'.format(np.sum(scipy_ans)))
print('DATA 원소들의 softmax 값 : {:.4f}, {:.4f}, {:.4f}'.format(*scipy_ans))

max: 3 VS argmax: 1
DATA 원소들의 softmax 전체 합 : 1.0
DATA 원소들의 softmax 값 : 0.0900, 0.2447, 0.6652
DATA 원소들의 softmax 전체 합 : 1.0
DATA 원소들의 softmax 값 : 0.0900, 0.6652, 0.2447


1. knowledge
- 의사 결정이 가장 중요한 초창기에 지식이 오히려 별로 없음, 불확실성이 큼
- logit / sigmoid / softmax 개념

2. knowledge distillation
- model에 있는 data가 무거우므로 증류하여 작은 network로 전달
- transfer learning 은 다른 domain 사이의 전달이라면 knowledge distillation은 같은 domain에서 model의 size를 작게 만들기 위한 전달

3. teacher-student loss & distillation
- pretrained teacher model로부터 input에 대해 soft label을 얻고, student(distilled) model로부터 input에 대해 soft prediction(확률 실수값)과 hard prediction(0 / 1)을 얻음
    - soft label 과 soft prediction 사이에 distillation loss
    - hard label(ground truth) 과 hard prediction 사이에 student loss
    - softened label : 확률 값 사이의 유사성도 학습에 사용하기 위함

4. zero-mean assumption
- teacher-student 를 통해 나온 distribution이 zero mean을 가질 때 각각의 logit에 따라 gradient를 정의하게 되고 model compression 이 knowledge distillation 의 특수한 경우임을 알 수 있음